## Illustration of predict_proba

based on: https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html#sklearn.naive_bayes.GaussianNB

In [16]:
import numpy as np
X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
Y = np.array([1, 1, 1, 2, 2, 2])

In [17]:
from sklearn.naive_bayes import GaussianNB

In [18]:
clf = GaussianNB()
clf.fit(X, Y)

GaussianNB(priors=None, var_smoothing=1e-09)

In [19]:
clf.predict([[-0.8, -1]])

array([1])

In [20]:
clf.predict_proba([[-0.8, -1]])

array([[9.99999949e-01, 5.05653254e-08]])

## Save and upload model

In [21]:
model_name='probabilities-example'

from waylay import WaylayClient, RestResponseError
waylay = WaylayClient.from_profile()
waylay.byoml.model.upload(
    model_name=model_name, 
    trained_model=clf, 
    framework='sklearn',
    description='Demo model sklearn probabilities.'
)

{'message': 'Model successfully uploaded'}

In [22]:
waylay.byoml.model.get(model_name)

{'createdBy': 'users/0476f99d-ac60-4f86-bc45-7215962754cd',
 'createdOn': '2021-04-27T13:50:44.309785+00:00',
 'description': 'Demo model sklearn probabilities.',
 'framework': 'sklearn',
 'name': 'probabilities-example',
 'supported_endpoints': ['predict'],
 'updatedBy': 'users/0476f99d-ac60-4f86-bc45-7215962754cd',
 'updatedOn': '2021-04-27T13:50:44.309838+00:00'}

## Check if model and uploaded model act the same

In [23]:
features = [[-0.8, -1]]

In [24]:
clf.predict(features)

array([1])

In [25]:
from tenacity import Retrying, stop_after_attempt, wait_fixed

for attempt in Retrying(stop=stop_after_attempt(10),wait=wait_fixed(5), reraise=True):
    with attempt:
        
        result= waylay.byoml.model.predict(model_name, features)

result

array([1])

In [26]:
clf.predict_proba(features)

array([[9.99999949e-01, 5.05653254e-08]])

In [30]:
## TODO: custom body parameters not supported yet (https://github.com/waylayio/waylay-py/issues/77)

# waylay.byoml.model.predict(model_name, features, body={'probabilities': True})

## workaround
from waylay.service import WaylayResource
class CustomModelResource(WaylayResource):
    actions = {
        'predict': {
            'method': 'POST',
            'url': '/models/{}/predict'
        }
    }
waylay.byoml._add_waylay_resource('model_x', CustomModelResource)
waylay.byoml.reconfigure()

result = waylay.byoml.model_x.predict(model_name, body={'instances': features, 'probabilities': True})
result.body['predictions']

[[0.9999999494346747, 5.056532536795868e-08]]

## Delete model

In [14]:
waylay.byoml.model.remove(model_name)

{'message': 'Model successfully deleted'}